<h1> Polars: Faster than Pandas <h1>

In [ ]:
# pip install polars
import polars as pl 

In [ ]:
# Create DataFrame
from datetime import*
timedf = pl.DataFrame(
    {
        "Interger" : [1,2,3],
        "Name": ['Linh','Binh','Thu'],
        'Date': [
                datetime(2004,3,20),
                datetime(2007,2,18),
                datetime(2005,11,29)
        ],
        'Mark': [6,7,8]
    }
)
timedf

In [ ]:
# Read file csv
df = pl.read_csv("./apple_quality.csv",ignore_errors = True,has_header=True)
# Read file parquet
pq = pl.read_parquet("./exam.parquet",columns=['Name','Mark'])
df

<h1> Expression :<h1> 
 select, filter, with_columns, group_by

In [ ]:
##  SELECT:
# 1. Define the DataFrame we want the data from.
# 2. Select the data that we need.
df.select(pl.col("*"))
df.select(['Size','Weight'])

# Select first and last row
df.head(2)
df.tail(2)

In [ ]:
## Sample

# Take a random sample
df.sample(5)

# Randomly select fraction of rows. 
df.sample(fraction=0.1) 

In [ ]:
## FILTER:
# use to create subset
# Filter: Extract rows that meet logical criteria
df.filter(pl.col("Weight")>=5)
df.filter((pl.col("Weight")>4) & (pl.col("Quality")=='good'))

In [ ]:
## GROUP BY
df2 = pl.DataFrame(
    {
        "a": ["a", "b", "a", "b", "c"],
        "b": [1, 2, 1, 3, 3],
        "c": [5, 4, 3, 2, 1],
    }
)
df2.group_by("a",maintain_order=True).len()
df2.group_by("a",maintain_order=True).agg(pl.col('b').sum().alias("Total_B"),pl.col("c").mean().alias("AVG_C"))
df2

In [ ]:
## WITH COLUMNS
# Adding new column
# Create a new column and get in a new variable
df3=df2.with_columns((pl.col('c')*2).alias("C x 2"))

# Create several columns
df4=df3.with_columns(
    [
    ((pl.col('b')+pl.col('C x 2')).alias('b+Cx2')),
    (pl.col('b').mean().alias("Mean of b"))
    ]
)
df4

# Add a column to indexed the row
df4.with_row_index()

In [ ]:
## SORT DATA
sorted_data = df.sort(['Weight','Crunchiness'],descending=[True,False])
sorted_data
df.filter(df['Weight'].is_unique())

In [ ]:
## HANDLING MISSING DATA

# drop null rows
df.drop_nulls()
# Replace null with a value
df.fill_null(5555)
# Other ll strategies are "backward", "min", "max", "mean", "zero" and "one"
df.fill_null(strategy='zero')

# Filling NaN by given value (NaN: Not a Number)
df.fill_nan(55)

<h1> Reshaping data, CONCAT, MERGING TABLE, COLUMNS <h1>

In [ ]:
# Merging DataFrames Combining multiple DataFrames
dff = pl.DataFrame(
    {
        "foo": [1, 2, 3],
        "bar": [6, 7, 8],
        "ham": ["a", "b", "c"],
    }
)
x = pl.Series("apple", [10, 20, 30])
print(dff.hstack([x]))


dff1 = pl.DataFrame(
    {
        "foo": [1, 2],
        "bar": [6, 7],
        "ham": ["a", "b"],
    }
)
dff2 = pl.DataFrame(
    {
        "foo": [3, 4],
        "bar": [8, 9],
        "ham": ["c", "d"],
    }
)
print(dff1.vstack(dff2))
dff2.select(pl.all().sum())

In [ ]:
#  Append columns of DataFrames
pl.concat([dff,pl.DataFrame(x)],how = 'horizontal')

In [ ]:
# Append rows of DataFrame
pl.concat([dff1,dff2])

<h1> Summarize Data <h1>

In [ ]:
df.schema

In [ ]:
df.describe()

In [ ]:
len(df)
df.height

In [ ]:
df.shape

In [ ]:
# Count number of rows with each unique value of variable
df['Quality'].value_counts()

In [ ]:
# of distinct values in a column
df['Quality'].n_unique()

<h1> Join Table <h1>

In [ ]:
import numpy as np
import polars as pl
df_join = pl.DataFrame(
    {
        "a": range(8),
        "b": np.random.rand(8),
        "d": [1, 2.0, float("nan"), float("nan"), 0, -5, -42, None],
    }
)

df_join2 = pl.DataFrame(
    {
        "x": range(8),
        "y": ["A", "A", "A", "B", "B", "C", "X", "X"],
    }
)
df_join3 = df_join.join(df_join2,left_on='a',right_on='x')
df_join3

In [ ]:
d_join = pl.DataFrame(
    {
        "foo": [1, 2, 3],
        "bar": [6.0, 7.0, 8.0],
        "ham": ["a", "b", "c"],
    }
)
other_join = pl.DataFrame(
    {
        "apple": ["x", "y", "z"],
        "ham": ["a", "b", "d"],
    }
)
join = d_join.join(other_join,on='ham',how = 'outer')
join

<h1> Truy vấn SQL dùng Polars <h1>

In [ ]:
# ctx = pl.SQLContext()
pokemon_data = pl.read_csv("https://gist.githubusercontent.com/ritchie46/cac6b337ea52281aa23c049250a4ff03/raw/89a957ff3919d90e6ef2d34235e6bf22304f3366/pokemon.csv")
pokemon_data.sample(3)

In [ ]:
ctx = pl.SQLContext(register_globals=True,eager_execution=False)
poke_small = ctx.execute(
    '''
    select *
    from pokemon_data
    order by HP desc
    where HP>115
    
    limit 3
    '''
)
poke_small.collect()